In [1]:
import pandas as pd   # 추출된 자료를 데이터 프레임 형태로 사용하기 위한 라이브러리 호출 (Call library for cursor to dataframe)


from pyhive import presto   # Hive와 연결하기 위한 라이브러리 호출 (Call library for connection to Hive)
from requests.auth import HTTPBasicAuth   # Hive 계정인증을 위한 라이브러리 호출

var_id = 'dopark41'   # Hive ID
var_ps = 'rbqlschlrh23###'   # Hive Password
var_db = 'hive'   # DataBase Name (변경할 필요없음 / No need to change)
var_url = 'cfs-presto.coupang.net'   # DataBase URL (변경할 필요없음 / No need to change)



# Connection Type 2

conn = presto.connect(host=var_url, port=443, protocol='https', catalog=var_db, username=var_id, password=var_ps)



cur = conn.cursor()   # Cursor for Query Sending to DB # 접속 커서를 만드는 코드


### Alternative Connection 
# 구동문

In [2]:
# Test Query
query =\
'''
SELECT floor, type, count(location) plt
FROM (WITH NYS as(SELECT INV.createdat
     , CEN.DESCRIPTION AS center
     , SKU.ID AS skuid
     , SKU.EXTERNALID externalid
     , CASE WHEN LOT.FULLNAME LIKE('108-RCRT0-1%') THEN 0.5
		 		WHEN LOT.FULLNAME LIKE('108-RCRT0-3%') THEN 1
		 		WHEN LOT.FULLNAME LIKE('108-RCRT0-5%') THEN 3
		 		WHEN LOT.FULLNAME LIKE('108-RCRT0-7%') THEN 4
		 		ELSE 0 END as floor
     , LOT.FULLNAME AS location
     , REPLACE(REPLACE(REPLACE(REPLACE(TRIM(SKU.NAME), CHR(9), ' '), CHR(10), ' '), CHR(13), ' '),'','') AS skuname
     , INV.QUANTITY AS qty
     , INV.allocatedquantity AS allocation
     , CASE WHEN INV.allocatedquantity > 0 THEN 'NYP'
     	 ELSE 'NYS' END AS type
FROM HIVE.ODS.INVENTORIES     INV
INNER JOIN HIVE.ODS.LOTS      LOT ON LOT.ID = INV.LOTID
INNER JOIN HIVE.ODS.LOCATIONS LOC ON LOC.ID = LOT.LOCATIONID
INNER JOIN HIVE.ODS.SKUS      SKU ON SKU.ID = INV.SKUID
INNER JOIN HIVE.ODS.CENTERS   CEN ON CEN.ID = LOT.CENTERID
WHERE 1 = 1
AND LOT.FULLNAME LIKE ('108-RCRT0-%')
AND LOT.FULLNAME NOT LIKE ('108-RCRT0-6%')
AND INV.ISUSABLE = 1
AND INV.QUANTITY > 0
AND CEN.ID IN (108) /* CENTER ID 입력 */
--AND SKU.ID IN (123456, 67890) /* 내부 SKUID 입력 - ## 외부 SKUID 로 작업시 밑에줄 사용 ## */
--AND CAST(SKU.EXTERNALID AS BIGINT) IN ()
ORDER BY 1,5
),
NYP as(SELECT INV.createdat
     , CEN.DESCRIPTION AS center
     , SKU.ID AS skuid
     , SKU.EXTERNALID externalid
     , CASE WHEN LOT.FULLNAME LIKE('108-0%') THEN 0.5
		 		WHEN LOT.FULLNAME LIKE('108-1%') THEN 1
		 		WHEN LOT.FULLNAME LIKE('108-3%') THEN 3
		 		WHEN LOT.FULLNAME LIKE('108-4%') THEN 4
		 		ELSE 0 END as floor
     , LOT.FULLNAME AS location
     , REPLACE(REPLACE(REPLACE(REPLACE(TRIM(SKU.NAME), CHR(9), ' '), CHR(10), ' '), CHR(13), ' '),'','') AS skuname
     , INV.QUANTITY AS qty
     , INV.allocatedquantity AS allocation
     , CASE WHEN INV.allocatedquantity > 0 THEN 'NYP'
     	 ELSE 'NYS' END AS type
FROM HIVE.ODS.INVENTORIES     INV
INNER JOIN HIVE.ODS.LOTS      LOT ON LOT.ID = INV.LOTID
INNER JOIN HIVE.ODS.LOCATIONS LOC ON LOC.ID = LOT.LOCATIONID
INNER JOIN HIVE.ODS.SKUS      SKU ON SKU.ID = INV.SKUID
INNER JOIN HIVE.ODS.CENTERS   CEN ON CEN.ID = LOT.CENTERID
WHERE 1 = 1
AND INV.	allocatedquantity >0 
AND INV.ISUSABLE = 1
AND INV.QUANTITY > 0
AND CEN.ID IN (108) /* CENTER ID 입력 */
--AND SKU.ID IN (123456, 67890) /* 내부 SKUID 입력 - ## 외부 SKUID 로 작업시 밑에줄 사용 ## */
--AND CAST(SKU.EXTERNALID AS BIGINT) IN ()
ORDER BY 1,5)
SELECT * FROM NYS
UNION	
SELECT * FROM NYP)
GROUP BY CUBE (floor, TYPE)
ORDER BY 1 DESC, 2, 3 
'''

cur.execute(query)   # Query를 실행 (Execute query)
df = pd.DataFrame(cur.fetchall(), columns=[ix[0] for ix in cur.description])  # 실행된 Query에서 추출된 자료를 DataFrame형태로 전달 (Send executed query to DataFrame)

conn.close()   # Close DataBase connection (추출완료 후, 연결종료를 실행해야 다른 프로그램에서 DataBase 접속이 가능해집니다) # 접속 종료 코드


In [3]:
df

,floor,type,plt
0,4.0,NYP,24
1,4.0,NYS,13
2,4.0,None,37
3,3.0,NYP,49
4,3.0,NYS,18
5,3.0,None,67
6,1.0,NYP,52
7,1.0,NYS,5
8,1.0,None,57
9,0.5,NYP,52
